In [1]:
import json
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

In [ ]:
# nltk.download('stopwords')

## Preprocessing

In [2]:
path_data_prepaired = '../dataset/dataset.json'

In [3]:
data = None
with open(path_data_prepaired) as file_data:
    data = json.load(file_data)

In [4]:
from preprocessing import clear_sentences

In [5]:
%time sentences = clear_sentences(data)

CPU times: user 4.38 s, sys: 304 ms, total: 4.68 s
Wall time: 4.69 s


In [ ]:
l = len(sentences)
sentences = sentences[:l//10]

## 1) Words Embeddings as Matrix Factorization

In [6]:
from models import Word2Vec

In [7]:
model = Word2Vec(sentences)

In [9]:
model.create_vocabulary()
model.create_corpus_matrix()

Creating vocabulary
Creating corpus matrix


In [ ]:
model.compute_embedds_IMF(5)

In [ ]:
model.compute_embedds_riem(5, max_iter=8)

In [10]:
model.compute_embedds_EMF(5, eps=5e-1, max_iter=50, step=1e-8)

0 iteration
2.046855502882254
2.046855502882254
2.046855502882253
2.0468555028773094
2.046855476346481
2.0466806284926182
1.7322746676365006
1.6248360879268595
1.6146931311168073
1.611917154517092
1.5892732264132385
1.3283955866911075
1.1954332804156924
1.1933436557833172
1.1930888411822205
1.191814587332711
1.174845623661564
1.127931553566037
1.11822433888567
1.11686996595726
1.1128242496763572
1.08979259235586
1.0322785039015099
0.9687721047495145
0.9406286622159562
0.9328165905672475
0.9272993664150063
0.9138942577804542
0.8877741647302183
0.8579076141746441
0.831888507501214
0.8053704813803598
0.7814097800568445
0.7634250076699038
0.7493195468240934
0.7374713237283224
0.7276184534391776
0.7199340837792481
0.7108868733865582
0.698128592029525
0.6847439485032899
0.6734531751542692
0.6641790923564923
0.6573335888099204
0.6531913338775093
0.6508746204793237
0.6493128422757493
0.6476784467482095
0.6453616912360648
0.642180615513088
First loop finished
Second loop finished
1 iteration
0.

KeyboardInterrupt: 

In [14]:
len(model.vocab)

3723

In [12]:
model.Ct.shape

(200, 3723)

In [11]:
model.W.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
##### Compute review embeddings #####
def get_review_embedding(model, review):
    """
    model -- word2vec model instance, which is used
    review -- current review to be embedded
    """
    
    review_vec = np.zeros(model.d)
    words_count = 0
    stops = set(stopwords.words("english"))
    
    for word in review:
        if (word in model.vocab) and not (word in stops):
            review_vec += model.get_word_embedding(word)
            words_count += 1
    review_vec /= words_count
    return review_vec

In [ ]:
##### Compute review embeddings #####
def get_features_matrix(model, reviews):
    """
    model -- word2vec model instance, which is used
    reviews -- the whole collection of reviews
    """
    X = np.zeros((len(reviews), model.d))
    for idx, review in enumerate(reviews):
        X[idx, :] = get_review_embedding(model, review)
    return X

In [15]:
X = model.get_features_matrix2(sentences)
X.shape

/Users/mikhail/projects/edu/skoltech/nla2018-master/project/NLA--project/notebooks/models.py:268: RuntimeWarning: invalid value encountered in true_divide
  review_vec /= words_count


(194439, 200)

### Comparison models of embeddings

In [27]:
# import clissifiers and necessary functions
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [17]:
# create dataframe from the json file
df = pd.read_json(path_data_prepaired)

In [18]:
# get labels
y = (df['overall'] > 3).apply(int)

In [19]:
# get indices of rows which contain NaNs
del_idx = np.argwhere(np.isnan(X))[:, 0]

In [20]:
# delete rows with NaNs
X = X[~np.isnan(X).any(axis=1)]
y = y.drop(del_idx)

In [21]:
y.shape, X.shape

((194286,), (194286, 200))

In [22]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

Try RF classifier out of the box

In [23]:
rf_clf = RandomForestClassifier(random_state=42)
#cv_scores = cross_val_score(rf_clf, X_train, y_train, n_jobs=-1, cv=5)

In [ ]:
print(f'Accuracy score: {cv_scores.mean()} +/- {cv_scores.std()}')

Evaluate model on the holdout set

In [25]:
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)


NameError: name 'accuracy' is not defined

In [28]:
print(f'Accuracy on holdout set: {f1_score(y_test, y_pred)}')

Accuracy on holdout set: 0.8329184726720551


In [ ]:
params = {'n_estimators': list(range(5, 51, 5)), 'max_depth': list(range(5, 100, 5)), 
         'min_samples_split': list(range(1, 11, 1))}

clf = RandomizedSearchCV(RandomForestClassifier(), params, n_jobs=-1, 
                         cv=5, verbose=1, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)
print(f'Accuracy on holdout set: {roc_auc_score(y_test, y_pred)}')

xgboost is very slow on mac ....

In [ ]:
xgbm_cls = XGBClassifier(random_state=42)
cv_scores = cross_val_score(xgbm_cls, X_train, y_train, n_jobs=-1, cv=5)

In [ ]:
print(f'Accuracy score: {cv_scores.mean()} +/- {cv_scores.std()}')

In [ ]:
X_df = pd.DataFrame(X)

In [ ]:
X_df.to_csv('../dataset/X_1.csv')
y.to_csv('../dataset/y_1.csv')

In [ ]:
X_df.to_csv('../dataset/X_1.csv')

In [ ]:
y.shape